In [23]:
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-10-29 22:46:36--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-10-29 22:46:36--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.2’

documents.json.2    100%[===================>] 642.90K  --.-KB/s    in 0.003s  

2024-10-29 22:46:36 (189 MB/s) - ‘documents.json.2’ saved [658

In [24]:
# !head documents.json

In [25]:
import json

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [26]:
#documents[2]

In [27]:
#len(documents)

In [28]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '41dab5d6d2e6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'SheWteUDQK-1StoAhxwPBA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/x5FRN2oNQmWZKIA79_7F6Q] already exists')

In [29]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [30]:
user_question = "How do I join the course after it has started?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [31]:
response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...



In [32]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [33]:
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...



In [34]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "The course already started. Can I still join?"}]
)
print(response.choices[0].message.content)

If the course has already started, your ability to join may depend on several factors, including the institution's policies, the instructor's flexibility, and how far along the course is. Here are some steps you can take:

1. **Contact the Instructor or Course Coordinator**: Reach out directly to the person in charge of the course to explain your situation. They may be able to offer you a late enrollment option or make accommodations.

2. **Check Enrollment Policies**: Look into the institution's policies regarding late enrollment to understand your options and any potential penalties or restrictions.

3. **Assess the Course Progress**: Consider how much of the course has already been completed and whether you can realistically catch up without negatively impacting your performance.

4. **Ask About Course Materials and Recordings**: Inquire if lecture recordings or materials from previous sessions are available to help you get up to speed.

5. **Consider Alternative Options**: If joini

In [35]:
context_template = """
Section: {section}
Question: {question}
Answer: {text}
""".strip()

context_docs = retrieve_documents(user_question)

context_result = ""

for doc in context_docs:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()
print(context)

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: G

In [36]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contain the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [37]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)
answer = response.choices[0].message.content
answer

"Yes, even if you don't register, you are still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it is advised not to leave everything for the last minute."

In [38]:
context_template = """
Section: {section}
Question: {question}
Answer: {text}
""".strip()

prompt_template = """
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()


def build_context(documents):
    context_result = ""
    
    for doc in documents:
        doc_str = context_template.format(**doc)
        context_result += ("\n\n" + doc_str)
    
    return context_result.strip()


def build_prompt(user_question, documents):
    context = build_context(documents)
    prompt = prompt_template.format(
        user_question=user_question,
        context=context
    )
    return prompt

def ask_openai(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = ask_openai(prompt)
    return answer

In [39]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")

'The error "invalid reference format: repository name must be lowercase" typically occurs when there is an issue with volume mapping in Docker, especially on Windows. Here are some solutions you can try according to the CONTEXT provided:\n\n1. **Move your data**: Ensure your data path does not include spaces. For example, move it from `C:/Users/Alexey Grigorev/git/...` to `C:/git/...`.\n\n2. **Replace the `-v` part with different options**: \n   - `-v /c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n   - `-v //c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n   - `-v /c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n   - `-v //c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data`\n   - `--volume //driveletter/path/ny_taxi_postgres_data/:/var/lib/postgresql/data`\n\n3. **Use `winpty`**: If you\'re using a Windows command line, try adding `winpty` before the Docker command:\n   ```shell\n   winpty docker run -it ...\n   ```\n\n4. **Add quotes*

In [40]:
qa_bot("I can't connect to postgres port 5432, my password doesn't work")

'It seems that your issue with connecting to Postgres on port 5432 and the password not working could be due to a few reasons mentioned in the CONTEXT:\n\n1. **Port Conflict**: The port 5432 might be occupied by another Postgres instance running on your machine. Try substituting port 5432 with another port, like 5431 or 8080, that you\'ve mapped during the Docker setup.\n\n2. **Windows Service Conflict**: If you\'re on Windows, check if there\'s a service running a separate Postgres instance that might be interfering. Stopping that service could resolve the issue.\n\n3. **User Role Issue**: If you encounter a "role \'root\' does not exist" error, make sure the root user exists in your Postgres setup. Otherwise, potential solutions include changing the user to `PGUSER=postgres` in your Docker Compose setup or recreating your Docker setup with a different configuration.\n\nPlease try these steps to diagnose and resolve your connection issue.'

In [41]:
qa_bot("how can I run kafka?")

'To run Kafka components like the producer, consumer, or kstreams in the terminal with Java, navigate to the project directory and execute the following command:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the appropriate name of your JAR file.'